# Part 2: Federated Learning入門

前回のセクションでは、PointerTensorについて学びました。PointerTensorは、プライバシーに配慮したディープラーニングを実現するための基礎となるツールで、インフラとしての役割をはたします。このセクションでは、それらのツールの使い方、特に私たちが最初に学ぶプライバシーに配慮したディープラーニングの手法である、Federated Learningの実装方法について学びます。

著者:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

### Federated Learningとは?

Federated Learningはディープラーニングのモデルを学習させるシンプル、かつパワフルな手法の一つです。

ディープラーニングの学習プロセスは、通常、データを集めるところから始まります。
データは人がパソコンやモバイルなどの端末を使って記録、生成することによって作成され、実際に学習プロセスを実行するマシンに集められます。
その後、集められたデータを使って学習処理を実行します。
これが、一般的なディープラーニングの学習プロセスです。

ですが、Federated Learningではこのプロセスをひっくり返します。

データをモデルのところへ持って行くのではなく、モデルをデータのあるところへ持って行くのです。
データの所有者はデータのコピーを誰にも渡さず、アクセス権限だけをコントロールするという考え方です。

クールでしょ?

# Section 2.1 - Federated Learningのトイモデル（初歩的な事例）

まずは、みなさんも良くご存知の（ごく普通の？）、データを一箇所に集めるやり方でのモデル学習を確認しましょう。
最小構成で以下の3つが必要です。

- トイ データセット
- モデル
- 基本的なトレーニングロジック

注記: もし、トイモデルを学習するPytorchのAPIについて解説が必要な方は、チュートリアルを進める前に[fast.ai](http://fast.ai)のコースを学習される事をおすすめします。

In [ ]:
import torch
from torch import nn
from torch import optim

In [ ]:
# トイデータセット（初歩的なモデル学習で使うデータ）
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# トイモデル（初歩的なモデル）
model = nn.Linear(2,1)

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) Gradientsの初期化 (もし、あれば)
        opt.zero_grad()

        # 2) 予測を行う
        pred = model(data)

        # 3) ロスの計算
        loss = ((pred - target)**2).sum()

        # 4) 勾配ベクトルの計算
        loss.backward()

        # 5) 勾配ベクトルの適用
        opt.step()

        # 6) 進捗を表示
        print(loss.data)

In [ ]:
train()

できました。今回は基本的なモデルを（従来からある）ごくごく一般的な手法で学習させてみました。全ての学習データはわたしたちのマシン上にあります。ローカルにあるデータセットを使ってローカルにあるモデルのパラメータをアップデートしています。しかし、Federatd Learningの世界では同じようにはいきません。
では、Federated Learning"的"な方法で同じモデルを学習させてみましょう。

必要なステップ:

- 複数のワーカー（ユーザーの事です）を作成する
- 作成したワーカーが保有するPointerTensorを取得する
- Federated Learning用にトレーニングロジックを調整する

    新しいトレーニングステップ（トレーニングロジック）:
    - モデルをデータセットを保持しているワーカーへ送る
    - モデルをデータセットのある場所で学習させる
    - モデルを返してもらい、次のワーカーに対して同様の処理を行う。ワーカーの数だけ繰り返す。
    

In [ ]:
import syft as sy
hook = sy.TorchHook(torch)


In [ ]:
# データセットを保持する複数のワーカーを作成する

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")


In [ ]:
# トイデータセット（初歩的なモデルで使うデータセット）
data = torch.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = torch.tensor([[0],[0],[1],[1.]], requires_grad=True)

# 学習データを分割し、半分をBob、残り半分をAliceに送り、それぞれのポインタを取得する
data_bob = data[0:2]
target_bob = target[0:2]

data_alice = data[2:]
target_alice = target[2:]

# トイモデル（初歩的なモデル）を初期化する
model = nn.Linear(2,1)

data_bob = data_bob.send(bob)
data_alice = data_alice.send(alice)
target_bob = target_bob.send(bob)
target_alice = target_alice.send(alice)

# BobとAliceのポインタをそれぞれ纏める
datasets = [(data_bob,target_bob),(data_alice,target_alice)]

opt = optim.SGD(params=model.parameters(),lr=0.1)

In [ ]:
def train():
    # トレーニングロジック
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(10):
        
        # NEW) ワーカーの数だけ繰り返す
        for data,target in datasets:
            
            # NEW) モデルを該当ワーカーへ送る
            model.send(data.location)

            # 1) 勾配ベクトルの初期化 (もし、あれば)
            opt.zero_grad()

            # 2) 予測する
            pred = model(data)

            # 3) ロスを計算する
            loss = ((pred - target)**2).sum()

            # 4) 勾配ベクトルを計算する
            loss.backward()

            # 5) 勾配ベクトルの適用
            opt.step()
            
            # NEW) モデルを返してもらう (計算済み勾配ベクトル含む)
            model.get()

            # 6) 進捗を表示
            print(loss.get()) # NEW) 微調整... .get()でデータを私たちのマシンに持ってくる必要があります。
    
# federated learning環境下での勾配ベクトルの集計作業 ＜ 後ほど解説します

In [ ]:
train()

## 良く出来ました！

今回は、トイモデル（初歩的なモデル）をFederated Learningの手法を使って学習してみました。モデルをそれぞれのワーカーへ送り、そこで勾配ベクトルの計算とモデルへの適用を行い、その後、学習済みモデルを私たちのマシンへ戻しています。この一連のプロセスにおいて、私たちは学習データ自体に対するアクセス権限をBobやAliceに要求していません。つまり、BobとAliceのプライバシーは保たれています！


## この例の不完全な部分

この例はFederated Learningの入門事例としては良いものの、いくつかの点で問題があります。その中でも最もよく知られた問題は、私たちが`model.get()`によって返してもらう学習済みのモデルの勾配ベクトルを調べる事で、実際にはBobやAliceのデータについて多くを知る事が可能だと言う事です。特定の状況下では、彼らのデータを完璧に復元することも可能です。

では、どうすれば良いでしょうか？まず思いつく手法は、私たちのマシンにモデルを返してもらう前に、複数のワーカーのgradientsを平均することです。しかし、そのためにはPointerTensorオブジェクトの高度な使い方を理解する必要があります。次のセクションでは、PointerTensorの発展的な機能について学び、今回学んだシンプルな例をアップデートしていきましょう。


# おめでとうございます！コミュニティに入ろう！

本チュートリアルを完了しました。おめでとうございます！もし、このチュートリアルを気に入って、プライバシーに配慮した非中央集権的なAI技術や付随する（データやモデルの）サプライチェーンにご興味があって、プロジェクトに参加したいと思われるなら、以下の方法で可能です。

### PySyftのGitHubレポジトリにスターをつける

一番簡単に貢献できる方法はこのGitHubのレポジトリにスターを付けていただくことです。スターが増えると露出が増え、より多くのデベロッパーにこのクールな技術の事を知って貰えます。

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Slackに入る

最新の開発状況のトラッキングする一番良い方法はSlackに入ることです。
下記フォームから入る事ができます。
[http://slack.openmined.org](http://slack.openmined.org)

### コードプロジェクトに参加する

コミュニティに貢献する一番良い方法はソースコードのコントリビューターになることです。PySyftのGitHubへアクセスしてIssueのページを開き、"Projects"で検索してみてください。参加し得るプロジェクトの状況を把握することができます。また、"good first issue"とマークされているIssueを探す事でミニプロジェクトを探すこともできます。

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### 寄付

もし、ソースコードで貢献できるほどの時間は取れないけど、是非何かサポートしたいという場合は、寄付をしていただくことも可能です。寄附金の全ては、ハッカソンやミートアップの開催といった、コミュニティ運営経費として利用されます。

[OpenMined's Open Collective Page](https://opencollective.com/openmined)